# Agents
By themselves, language models can't take actions - they just output text. A big use case for LangChain is creating agents. 
Agents are systems that use an LLM as a reasoning enginer to determine which actions to take and what the inputs to those actions should be. 
The results of those actions can then be fed back into the agent and it determine whether more actions are needed, or whether it is okay to finish.

In [3]:
# Import relevant functionality
import os
import openai
from dotenv import load_dotenv
from langchain_openai import AzureChatOpenAI
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain_core.messages import HumanMessage
from langgraph.checkpoint.sqlite import SqliteSaver
from langgraph.prebuilt import create_react_agent

# Load environment variables (set OPENAI_API_KEY and OPENAI_API_BASE in .env)
load_dotenv()

# Configure Azure OpenAI Service API
openai.api_type = "azure"
openai.api_version = os.getenv("AZURE_OPENAI_VERSION")
openai.api_base = os.getenv('OPENAI_API_ENDPOINT')
openai.api_key = os.getenv("OPENAI_API_KEY")



In [4]:
# Create the Azure OpenAI model
llm = AzureChatOpenAI(
    deployment_name="gpt4o",  # replace with your deployment name
    temperature=0.5,
    max_tokens=100,
    openai_api_version="2023-03-15-preview"
)

# Create the search tool
search = TavilySearchResults(max_results=2)
tools = [search]

# Create the agent with memory
memory = SqliteSaver.from_conn_string(":memory:")
agent_executor = create_react_agent(llm, tools, checkpointer=memory)

# Configuration for conversation thread
config = {"configurable": {"thread_id": "abc123"}}



In [5]:
# testing the agent with a few messages

for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="load the titanic.csv into a pandas dataframe")]}, config
):
    print(chunk)
    print("----")
    
    
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="how many rows are there in the titanic.csv?")]}, config
):
    print(chunk)
    print("----")
    
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="tell me more about the passengers")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Sure, I can help you with that. However, I don't have direct access to files like `titanic.csv`. If you provide the contents of the file or a link to it, I can guide you on how to load it into a pandas DataFrame. Here's a general example of how you can load a CSV file into a pandas DataFrame:\n\n```python\nimport pandas as pd\n\n# Load the CSV file into a DataFrame\ndf = pd.read_csv('path/to/t", response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 88, 'total_tokens': 188}, 'model_name': 'gpt-4o-2024-05-13', 'system_fingerprint': 'fp_5f4bad809a', 'finish_reason': 'length', 'logprobs': None, 'content_filter_results': {}}, id='run-028e9436-8049-483d-a6eb-1f9f6701de61-0', usage_metadata={'input_tokens': 88, 'output_tokens': 100, 'total_tokens': 188})]}}
----
{'agent': {'messages': [AIMessage(content="To determine the number of rows in the `titanic.csv` file, you would typically load the file into a pandas DataFrame and the

In [8]:
from openai import AzureOpenAI
import requests
import time

# Initialize the client
client = AzureOpenAI(
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    api_version="2024-05-01-preview",
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT")
)

# Example function to list assistants
def list_assistants():
    response = client.beta.assistants.list(order="desc", limit=20)
    return response.data

# Example function to interact with an assistant
def interact_with_assistant(assistant_id, message):
    thread = client.beta.threads.create(assistant_id=assistant_id)
    client.beta.threads.messages.create(
        thread_id=thread.id,
        content=[{"type": "text", "text": {"value": message}}]
    )
    run = client.beta.threads.runs.create(thread_id=thread.id)
    
    while run.status not in ["completed", "cancelled", "expired", "failed"]:
        time.sleep(5)
        run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    
    messages = client.beta.threads.messages.list(thread_id=thread.id)
    return messages

# List assistants
assistants = list_assistants()
print(assistants)

# Interact with a specific assistant
assistant_id = "your_assistant_id"
response = interact_with_assistant(assistant_id, "What's the weather in San Francisco?")
print(response)

[]


TypeError: Threads.create() got an unexpected keyword argument 'assistant_id'